In [5]:
import pandas as pd
import dotenv
dotenv.load_dotenv()

True

In [6]:
df_london_smartmeter = pd.read_csv("./data/LondonSmartMeter/london_smart_meters_dataset_without_missing_values_first_30_consumers.csv", index_col='Time', parse_dates=['Time'])
df_kddcup = pd.read_csv("./data/KDDCup_2018/kdd_cup_2018_dataset_without_missing_values.csv", index_col='Time', parse_dates=['Time'])
df_electricity_321_hourly = pd.read_csv("./data/Electricity321Hourly/electricity_hourly_dataset.csv", index_col='Time', parse_dates=['Time'])
df_electricity_370 = pd.read_csv("./data/Electricity370/LD2011_2014_first_40_consumers.csv", index_col='Time', parse_dates=['Time'])
# tno_df.head()
# print(df.columns.tolist()[:1])

In [7]:
import torch
import psutil

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the current device
    torch.set_float32_matmul_precision('high')
    device = torch.cuda.current_device()
    print(f"Using CUDA device: {torch.cuda.get_device_name(device)}")
else:
    device = 'cpu'
    print("CUDA is not available")

print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores:", psutil.cpu_count(logical=True))
cpu_freq = psutil.cpu_freq()
print(f"Current Frequency: {cpu_freq.current:.2f}Mhz")

# RAM Information
svmem = psutil.virtual_memory()
print(f"Total: {svmem.total / (1024 ** 3):.2f} GB")
print(f"Available: {svmem.available / (1024 ** 3):.2f} GB")
print(f"Used: {svmem.used / (1024 ** 3):.2f} GB")
print(f"Percentage: {svmem.percent}%")

print("Distributed PyTorch available:", torch.distributed.is_available())

Using CUDA device: NVIDIA GeForce RTX 3090
Physical cores: 64
Total cores: 128
Current Frequency: 2.75Mhz
Total: 1007.71 GB
Available: 403.55 GB
Used: 595.49 GB
Percentage: 60.0%
Distributed PyTorch available: True


In [8]:
from copy import deepcopy
from ts_inverse.models import FCN_Predictor, CNN_Predictor, GRU_Predictor, JitGRU_Predictor, CNNJitGRU_Predictor, TCN_Predictor, JitSeq2Seq_Predictor
from ts_inverse.utils import grid_search_params
from ts_inverse.workers import AttackTSInverseWorker


def start_multi_process(g_config, a_config, d_config, m_config, pool_size):
    search_args = []
    search_configs = list(grid_search_params(g_config))
    search_attack_configs = list(grid_search_params(a_config))
    search_dataset_settings = list(grid_search_params(d_config))
    search_model_settings = list(grid_search_params(m_config))
    for original_g_config in search_configs:
        for a_config in search_attack_configs:
            g_config = deepcopy(original_g_config)
            g_config.update(a_config)
            for m_config in search_model_settings:
                for d_config in search_dataset_settings:
                    fa_models_config = {
                        'features': [[0]],
                        'input_size': d_config['observation_days'],
                        'output_size': d_config['future_days'],
                    }
                    search_for_all_models_settings = list(grid_search_params(fa_models_config))
                    for fa_models_config in search_for_all_models_settings:
                        g_config['run_number'] = len(search_args)
                        args = (g_config, d_config, m_config, fa_models_config)
                        search_args.append(deepcopy(args))

    print(f"Starting {len(search_args)} processes")
    if pool_size == 1:
        for args in search_args:
            AttackTSInverseWorker(args[0]['run_number']).worker_process(*args)


global_config = {
    'logger_service': 'wandb',
    'experiment_name': 'ts-inverse_ts_regularization_11-6-2024',
    'seed': [10, 43], # 28, 80, 71],
    'batch_size': 4,
    'device': 1,
    'verbose': False,
    'pool_size': 1,
    'run_number': -1,
    'total_variation_alpha_inputs': 0, 
    'total_variation_beta_targets': 0,
    'after_effect': 'none',
    'warmup_number_of_batches': 0,
    'number_of_batches': 1,
    'update_model': False, # Update the model in generating gradients from training data
    'model_evaluation_during_attack': False, # Baselines do not consider this
    'load_lti_model': False,

    'dropout': 0,
    'optimize_dropout': False,
    'dropout_probability_regularizer': 0,
    'dummy_init_method': 'rand',
}

attack_config = [
    {
        'attack_method': 'TS-Inverse',
        # invert attack
        'num_learn_epochs': 0,
        'learn_learning_rate': 1e-3, 
        'attack_batch_size': 32,
        'inversion_batch_size': 1,
        'attack_hidden_size': [[768, 512]],
        'quantiles': [[0.1, 0.3, 0.7, 0.9]],
        'attack_loss': ['quantile'],
        'inversion_model': 'ImprovedGradToInputNN_Quantile', 
        'attack_targets': True,
        'learn_optimizer': 'adamW',
        'learn_lr_decay': ['on_plateau'],
        'aux_dataset': None,

        ## Inversion regularization in optimization attack
        'inversion_regularization_term_inputs': [0], 
        'inversion_regularization_term_targets': [0],
        'inversion_regularization_loss': ['quantile'],

        'lower_res_term': [0],
        'trend_term': [0],
        'trend_loss': ['l1_mean'],
        'trend_reduce_lr': [False],
        'periodicity_term': [0, 2, 1, 0.5, 0.1],
        'periodicity_loss': ['l1_mean', 'l2_mean'],
        'periodicity_reduce_lr': [False],


        ## Optimization attack
        'gradient_loss': 'l1', 
        'base_num_attack_steps': 500,
        'after_effect': 'clamp_2',
        'optimization_learning_rate': 0.01, 
        'attack_opti_optimizer': ['adam'],
        'attack_opti_lr_decay': ['on_plateau_10'],
        'optimize_dropout': True,
        'clamp_dropout': 1,
        'clamp_dropout_min_max': [[0.0, 1.0]],
        'dropout_probability_regularizer': [1e-5], #1e-6, 1e-7, 1e-8],
        'dropout_mask_init_type': 'halves', #['bernoulli', 'halves', 'uniform', 'p', '1-p'], #['halves', 'bernoulli'],
        'grad_signs_for_inputs': True,
        'grad_signs_for_targets': False,
        'grad_signs_for_dropouts': True, #[False, True],
    },
]

dataset_config = [
    {
        'dataset': 'electricity_370',
        'columns': [df_electricity_370.columns.tolist()[4:5]],
        'train_stride': 24,
        'validation_stride': 1,
        'observation_days': 1,
        'future_days': 1,
        'normalize': 'minmax',
    },
    {
        'dataset': 'london_smartmeter',
        'columns': [df_london_smartmeter.columns.tolist()[:1]],
        'train_stride': 24, # Is the strides that are attacked
        'validation_stride': 1, # Is the stride that is used for training the invertion model
        'observation_days': 1,
        'future_days': 1,
        'normalize': 'minmax',
    },
    # {
    #     'dataset': 'kddcup',
    #     'columns': [df_kddcup.columns.tolist()[:1]],
    #     'train_stride': 24,
    #     'validation_stride': 1,
    #     'observation_days': 5,
    #     'future_days': 2,
    #     'normalize': 'minmax',
    # },    
]

model_config = [
    # {
    #     '_model': FCN_Predictor,
    #     'hidden_size': 64,
    #     '_attack_step_multiplier': 10,
    # },
    # {
    #     '_model': CNN_Predictor,
    #     'hidden_size': 64,
    #     '_attack_step_multiplier': 10,
    # },
    {
        '_model': TCN_Predictor,
        'hidden_size': 64,
        'num_levels': 0,
        'kernel_size': 6,
        'dilation_factor': 2,
        'activation': 'relu',
        'use_weight_norm': True,
        'init_weights': True,
        'dropout': 0.1,
        '_attack_step_multiplier': 10,
    },
    # {
    #     '_model': JitGRU_Predictor,
    #     'hidden_size': 64,
    #     '_attack_step_multiplier': 10,
    # },
    # {
    #     '_model': JitSeq2Seq_Predictor,
    #     'hidden_size': 64,
    #     '_attack_step_multiplier': 10,
    # }
]

start_multi_process(global_config, attack_config, dataset_config, model_config, global_config['pool_size'])

Starting 60 processes


Starting attack 0 with config: {'logger_service': 'wandb', 'experiment_name': 'ts-inverse_ts_regularization_11-6-2024', 'seed': 10, 'batch_size': 4, 'device': 1, 'verbose': False, 'pool_size': 1, 'run_number': 0, 'total_variation_alpha_inputs': 0, 'total_variation_beta_targets': 0, 'after_effect': 'clamp_2', 'warmup_number_of_batches': 0, 'number_of_batches': 1, 'update_model': False, 'model_evaluation_during_attack': False, 'load_lti_model': False, 'dropout': 0.1, 'optimize_dropout': True, 'dropout_probability_regularizer': 1e-05, 'dummy_init_method': 'rand', 'attack_method': 'TS-Inverse', 'num_learn_epochs': 0, 'learn_learning_rate': 0.001, 'attack_batch_size': 32, 'inversion_batch_size': 1, 'attack_hidden_size': [768, 512], 'quantiles': [0.1, 0.3, 0.7, 0.9], 'attack_loss': 'quantile', 'inversion_model': 'ImprovedGradToInputNN_Quantile', 'attack_targets': True, 'learn_optimizer': 'adamW', 'learn_lr_decay': 'on_plateau', 'aux_dataset': None, 'inversion_regularization_term_inputs': 0, 